<a href="https://colab.research.google.com/github/talhaanwarch/HASOC20/blob/master/HASOC20_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install focal-loss
!pip install keras-tcn
!pip install tensorflow==2.1.0


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Import lib

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential,Model

from tensorflow.keras.layers import Embedding,GRU,LSTM,Dense,Dropout,Bidirectional,BatchNormalization,GlobalMaxPooling1D,Flatten, GlobalAveragePooling1D, MaxPooling1D,SpatialDropout1D,Input,Activation,concatenate,Conv1D,multiply
from tensorflow.keras.optimizers import RMSprop,Adam,Adadelta
from tensorflow.keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
# import keras
import collections
import numpy as np
from focal_loss import BinaryFocalLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from tensorflow.keras import backend as K
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
from tensorflow.keras.layers import Layer,Lambda,InputSpec
from tensorflow.keras import initializers
from tensorflow.keras import backend as K
# import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load data

In [4]:
train=pd.read_csv( '/content/drive/My Drive/dataset/HASOC20/hasoc_2020_en_train.csv')
train.head()

,tweet_id,text,task1,task2,ID
0,1123757263427186690,"hate wen females hit ah nigga with tht bro 😂😂,...",HOF,PRFN,hasoc_2020_en_2574
1,1123733301397733380,RT @airjunebug: When you're from the Bay but y...,HOF,PRFN,hasoc_2020_en_3627
2,1123734094108659712,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NOT,NONE,hasoc_2020_en_3108
3,1126951188170199049,RT @SheLoveTimothy: He ain’t on drugs he just ...,NOT,PRFN,hasoc_2020_en_3986
4,1126863510447710208,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NOT,NONE,hasoc_2020_en_5152


In [5]:
train_labels=train['task1']
train=train['text']

In [6]:
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')


In [7]:
train.head()

0    hate wen females hit ah nigga with tht bro 😂😂,...
1    RT @airjunebug: When you're from the Bay but y...
2    RT @DonaldJTrumpJr: Dear Democrats: The Americ...
3    RT @SheLoveTimothy: He ain’t on drugs he just ...
4    RT @TavianJordan: Summer ‘19 I’m coming for yo...
Name: text, dtype: object

In [11]:
import re
train=train.str.replace('\d+', '')
train= train.str.lower()
train=train.str.replace('rt.*: ','',flags=re.DOTALL)

train = train.str.replace('[^\w\s]','')
#train = train.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
#train = train.apply(lambda x: [item for item in x if item not in stop])
#train = train.apply(lambda x : " ".join(x))
train.head()

0    hate wen females hit ah nigga with tht bro  im...
1    when youre from the bay but youre really a ny ...
2    the american people arent stupid they know wha...
3    he aint on drugs he just bored i be doing the ...
4    summer  im coming for you  no boring shit  bea...
Name: text, dtype: object

#Label Encoding

In [13]:
le=LabelEncoder()
labels=le.fit_transform(train_labels)
print(len(labels))

3794


In [15]:
train_labels.value_counts()

HOF    1932
NOT    1862
Name: task1, dtype: int64

# Common Parameters

In [16]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

# Tokenize data

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 150

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len,padding='post')

In [18]:
len(sequences_matrix),len(labels)

(3794, 3794)

In [19]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [ ]:
# test_sequences = tokenize.texts_to_sequences(test)
# X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len,padding='post')

# Embeddings

In [20]:
embedding_path1 = "drive/My Drive/dataset/OLID/wiki-news-300d-1M.vec"
embedding_path2 = "drive/My Drive/dataset/OLID/glove/glove.840B.300d.txt"
embed_size = 300

In [21]:
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [22]:
# fasttext=build_matrix(embedding_path1, word_index)
# glove_emb=build_matrix(embedding_path2, word_index)

In [23]:
# embeddings=np.mean((fasttext,glove_emb),axis=0)
# embeddings.shape

In [24]:
#np.save('/content/drive/My Drive/dataset/OffenseEval2020/data/English/en_embeddings.npy',embeddings)

In [25]:
embeddings=np.load('/content/drive/My Drive/dataset/OffenseEval2020/data/English/en_embeddings.npy')
embeddings.shape

(10001, 300)

# Base Models

In [26]:
train.shape,labels.shape

((3794,), (3794,))

## Count Vectorizer

In [27]:

def logistic_param_selection(X, y, func,nfolds):
    C= [7,8,9,10,12,15,20,25]
    param_grid = {'C': C}
    grid_search = GridSearchCV(make_pipeline(func, LogisticRegression(solver='lbfgs',max_iter=500,class_weight=class_weights)),
                    param_grid={'logisticregression__C':C}, cv=nfolds,scoring='f1_macro')
    grid_search.fit(X, y)
    return grid_search.best_score_,grid_search.best_params_



In [28]:
cv_score,c=logistic_param_selection(train,labels,CountVectorizer(),5)
cv_score

0.8540543614027971

In [30]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train)
X_train_cv = count_vectorizer.transform(train)
#X_test_cv=  count_vectorizer.transform(test)

In [31]:
cv_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500,class_weight=class_weights)
cv_classifier.fit(X_train_cv, labels)

LogisticRegression(C=7,
                   class_weight={0: 0.9818840579710145, 1: 1.018796992481203},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=500, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [ ]:
#cv_test=cv_classifier.predict_proba(X_test_cv)[:,1]

## TF IDF word vectorizer

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=10000)
word_vectorizer.fit(train)
train_word_features = word_vectorizer.transform(train)
#test_word_features = word_vectorizer.transform(test)

In [36]:
tfw_score,c=logistic_param_selection(train,labels,word_vectorizer,5)
tfw_score

0.845548565812291

In [ ]:
tfw_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500)
tfw_classifier.fit(train_word_features, labels)


In [ ]:
tfw_test = tfw_classifier.predict_proba(test_word_features)[:,1]

## TF IDF char vectorizer

In [37]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000)
char_vectorizer.fit(train)
train_char_features = char_vectorizer.transform(train)
#test_char_features = char_vectorizer.transform(test)

In [38]:
tfc_score,c=logistic_param_selection(train,labels,char_vectorizer,5)
tfc_score

0.8540530522045847

In [ ]:
tfc_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500)
tfc_classifier.fit(train_char_features, labels)


LogisticRegression(C=7, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
tfc_test = tfc_classifier.predict_proba(test_char_features)[:,1]

# Models

##GRU

In [39]:
def model_gru():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(LSTM(50, return_sequences=True))(x)
  x, x_h, x_c = Bidirectional(GRU(50, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,reset_after=True,return_sequences = True, return_state = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, x_h, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

In [40]:
def model_gru1():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.2)(x)
  x=Bidirectional(GRU(150,activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,reset_after=True,return_sequences = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, max_pool])


  x=Dropout(0.1)(conc)
  out=Dense(64, activation="relu")(x)
  out=Dense(32, activation="relu")(x)

  outp=Dense(1, activation="sigmoid")(x)   
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

## GRU attention

In [41]:
#https://github.com/zake7749/DeepToxic


class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weight = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [42]:
def model_gru_attn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(LSTM(50, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,return_sequences = True))(x)
  x = Bidirectional(GRU(50, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,reset_after=True,return_sequences = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  last = Lambda(lambda t: t[:, -1])(x)
  attn = AttentionWeightedAverage()(x)
  conc = concatenate([avg_pool,  max_pool,last,attn])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

##TCN

In [43]:
from tcn import TCN

def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return multiply([tanh_out, sigm_out])

def model_tcn(embedding_matrix):
    
    inp = Input(shape=(max_len,))
    x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True ,name = 'tnc1')(x)
    x = wave_net_activation(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, name = 'tnc2')(x)
    x = wave_net_activation(x)
    #x = TCN(32,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc3')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dense(32, activation="relu")(conc)

    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
    
    return model

##KIM

In [44]:
conv_filters=128
def model_kim():
  inp = Input(shape=(max_len,))
  emb = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  # Specify each convolution layer and their kernel siz i.e. n-grams 
  conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(emb)
  btch1_1 = BatchNormalization()(conv1_1)
  drp1_1  = Dropout(0.2)(btch1_1)
  actv1_1 = Activation('relu')(drp1_1)
  glmp1_1 = GlobalMaxPooling1D()(actv1_1)

  conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(emb)
  btch1_2 = BatchNormalization()(conv1_2)
  drp1_2  = Dropout(0.2)(btch1_2)
  actv1_2 = Activation('relu')(drp1_2)
  glmp1_2 = GlobalMaxPooling1D()(actv1_2)

  conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(emb)
  btch1_3 = BatchNormalization()(conv1_3)
  drp1_3  = Dropout(0.2)(btch1_3)
  actv1_3 = Activation('relu')(drp1_3)
  glmp1_3 = GlobalMaxPooling1D()(actv1_3)

  conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(emb)
  btch1_4 = BatchNormalization()(conv1_4)
  drp1_4  = Dropout(0.2)(btch1_4)
  actv1_4 = Activation('relu')(drp1_4)
  glmp1_4 = GlobalMaxPooling1D()(actv1_4)

  # Gather all convolution layers
  cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
  drp1 = Dropout(0.2)(cnct)

  dns1  = Dense(32, activation='relu')(drp1)
  btch1 = BatchNormalization()(dns1)
  drp2  = Dropout(0.2)(btch1)

  out = Dense(1, activation='sigmoid')(drp2)   
  model = Model(inputs=inp, outputs=out)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model


##RNN_CNN

In [45]:

def model_gru_cnn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.5)(x)
  x = Bidirectional(LSTM(128, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,return_sequences = True))(x)
  x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  x = concatenate([avg_pool, max_pool])
  x = Dense(10, activation = "relu")(x)

  outp=Dense(1, activation="sigmoid")(x)   
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model


# Train model

In [58]:
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
import tensorflow as tf
clr=ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=3, min_lr=0.000001)

##GRU

In [59]:
sequences_matrix.shape,labels.shape

((3794, 150), (3794,))

In [60]:

kfold = StratifiedKFold(n_splits=5, shuffle=False)
gru_scores = []
gru_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  gru_model=model_gru()
  
  gru_model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

 #/ gru_test.append(gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  gru_scores.append(f1)
  tf.keras.backend.clear_session()


Train on 3035 samples, validate on 759 samples
Epoch 1/10
3035/3035 - 8s - loss: 0.1698 - accuracy: 0.5671 - val_loss: 0.1618 - val_accuracy: 0.6034
Epoch 2/10
3035/3035 - 2s - loss: 0.1548 - accuracy: 0.6583 - val_loss: 0.1434 - val_accuracy: 0.6877
Epoch 3/10
3035/3035 - 2s - loss: 0.1368 - accuracy: 0.7203 - val_loss: 0.1299 - val_accuracy: 0.7510
Epoch 4/10
3035/3035 - 2s - loss: 0.1214 - accuracy: 0.7634 - val_loss: 0.1193 - val_accuracy: 0.7734
Epoch 5/10
3035/3035 - 2s - loss: 0.1091 - accuracy: 0.8000 - val_loss: 0.1123 - val_accuracy: 0.7931
Epoch 6/10
3035/3035 - 2s - loss: 0.1009 - accuracy: 0.8244 - val_loss: 0.1097 - val_accuracy: 0.8076
Epoch 7/10
3035/3035 - 2s - loss: 0.0901 - accuracy: 0.8468 - val_loss: 0.1095 - val_accuracy: 0.8155
Epoch 8/10
3035/3035 - 2s - loss: 0.0774 - accuracy: 0.8751 - val_loss: 0.1126 - val_accuracy: 0.8024
Epoch 9/10
3035/3035 - 2s - loss: 0.0700 - accuracy: 0.8956 - val_loss: 0.1200 - val_accuracy: 0.8037
Epoch 10/10
3035/3035 - 2s - loss: 

In [61]:
np.mean(gru_scores)

0.7934497667600902

##GRU 1

In [62]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
gru_scores1 = []
gru_test1=[]
for train, val in kfold.split(sequences_matrix, labels):
  gru_model=model_gru1()
  
  gru_model.fit(sequences_matrix[train], labels[train],batch_size=128,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]))
  y_pred = gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  #gru_test1.append(gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  gru_scores1.append(f1)
  tf.keras.backend.clear_session()

Train on 3035 samples, validate on 759 samples
Epoch 1/10
3035/3035 - 4s - loss: 0.1683 - accuracy: 0.5783 - val_loss: 0.1567 - val_accuracy: 0.6640
Epoch 2/10
3035/3035 - 1s - loss: 0.1503 - accuracy: 0.6896 - val_loss: 0.1430 - val_accuracy: 0.7167
Epoch 3/10
3035/3035 - 1s - loss: 0.1389 - accuracy: 0.7249 - val_loss: 0.1338 - val_accuracy: 0.7418
Epoch 4/10
3035/3035 - 1s - loss: 0.1254 - accuracy: 0.7697 - val_loss: 0.1257 - val_accuracy: 0.7563
Epoch 5/10
3035/3035 - 1s - loss: 0.1154 - accuracy: 0.8000 - val_loss: 0.1196 - val_accuracy: 0.7813
Epoch 6/10
3035/3035 - 1s - loss: 0.1058 - accuracy: 0.8264 - val_loss: 0.1153 - val_accuracy: 0.7800
Epoch 7/10
3035/3035 - 1s - loss: 0.0965 - accuracy: 0.8458 - val_loss: 0.1110 - val_accuracy: 0.8037
Epoch 8/10
3035/3035 - 1s - loss: 0.0888 - accuracy: 0.8662 - val_loss: 0.1072 - val_accuracy: 0.8195
Epoch 9/10
3035/3035 - 1s - loss: 0.0809 - accuracy: 0.8830 - val_loss: 0.1095 - val_accuracy: 0.8076
Epoch 10/10
3035/3035 - 1s - loss: 

In [63]:
np.mean(gru_scores1)

0.8130888015447051

## GRU attention

In [64]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
atten_scores = []
atten_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  attn_gru_model=model_gru_attn()
  
  attn_gru_model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = attn_gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  #atten_test.append(attn_gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  atten_scores.append(f1)
  tf.keras.backend.clear_session()

Train on 3035 samples, validate on 759 samples
Epoch 1/10
3035/3035 - 8s - loss: 0.1693 - accuracy: 0.5647 - val_loss: 0.1583 - val_accuracy: 0.6482
Epoch 2/10
3035/3035 - 2s - loss: 0.1529 - accuracy: 0.6649 - val_loss: 0.1457 - val_accuracy: 0.6838
Epoch 3/10
3035/3035 - 2s - loss: 0.1349 - accuracy: 0.7292 - val_loss: 0.1316 - val_accuracy: 0.7325
Epoch 4/10
3035/3035 - 2s - loss: 0.1227 - accuracy: 0.7624 - val_loss: 0.1199 - val_accuracy: 0.7708
Epoch 5/10
3035/3035 - 2s - loss: 0.1072 - accuracy: 0.8119 - val_loss: 0.1148 - val_accuracy: 0.7918
Epoch 6/10
3035/3035 - 2s - loss: 0.0974 - accuracy: 0.8300 - val_loss: 0.1140 - val_accuracy: 0.7747
Epoch 7/10
3035/3035 - 2s - loss: 0.0872 - accuracy: 0.8491 - val_loss: 0.1087 - val_accuracy: 0.8155
Epoch 8/10
3035/3035 - 2s - loss: 0.0791 - accuracy: 0.8712 - val_loss: 0.1466 - val_accuracy: 0.7233
Epoch 9/10
3035/3035 - 2s - loss: 0.0705 - accuracy: 0.8840 - val_loss: 0.1211 - val_accuracy: 0.7971
Epoch 10/10
3035/3035 - 2s - loss: 

In [65]:
 np.mean(atten_scores)

0.794886579582516

##TCN

In [66]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
tcn_scores = []
tcn_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_tcn(embeddings)
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  #tcn_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  tcn_scores.append(f1)
  tf.keras.backend.clear_session()

Train on 3035 samples, validate on 759 samples
Epoch 1/10
3035/3035 - 4s - loss: 0.1753 - accuracy: 0.5249 - val_loss: 0.1707 - val_accuracy: 0.5599
Epoch 2/10
3035/3035 - 1s - loss: 0.1688 - accuracy: 0.5756 - val_loss: 0.1659 - val_accuracy: 0.5863
Epoch 3/10
3035/3035 - 1s - loss: 0.1432 - accuracy: 0.6873 - val_loss: 0.1297 - val_accuracy: 0.7233
Epoch 4/10
3035/3035 - 1s - loss: 0.1243 - accuracy: 0.7644 - val_loss: 0.1157 - val_accuracy: 0.7787
Epoch 5/10
3035/3035 - 1s - loss: 0.1100 - accuracy: 0.8046 - val_loss: 0.1225 - val_accuracy: 0.7628
Epoch 6/10
3035/3035 - 1s - loss: 0.0923 - accuracy: 0.8524 - val_loss: 0.1050 - val_accuracy: 0.8274
Epoch 7/10
3035/3035 - 1s - loss: 0.0730 - accuracy: 0.8942 - val_loss: 0.1113 - val_accuracy: 0.8182
Epoch 8/10
3035/3035 - 1s - loss: 0.0605 - accuracy: 0.9160 - val_loss: 0.1249 - val_accuracy: 0.8169
Epoch 9/10
3035/3035 - 1s - loss: 0.0481 - accuracy: 0.9318 - val_loss: 0.1230 - val_accuracy: 0.8208
Epoch 10/10
3035/3035 - 1s - loss: 

In [67]:
np.mean(tcn_scores)

0.8145217588362555

##KIM

In [68]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
kim_scores = []
kim_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_kim()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  #kim_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  kim_scores.append(f1)
  tf.keras.backend.clear_session()

Train on 3035 samples, validate on 759 samples
Epoch 1/10
3035/3035 - 3s - loss: 0.2969 - accuracy: 0.5453 - val_loss: 0.2189 - val_accuracy: 0.5099
Epoch 2/10
3035/3035 - 1s - loss: 0.1710 - accuracy: 0.6639 - val_loss: 0.2011 - val_accuracy: 0.5099
Epoch 3/10
3035/3035 - 1s - loss: 0.1465 - accuracy: 0.7321 - val_loss: 0.1987 - val_accuracy: 0.5099
Epoch 4/10
3035/3035 - 1s - loss: 0.1193 - accuracy: 0.7954 - val_loss: 0.1965 - val_accuracy: 0.5112
Epoch 5/10
3035/3035 - 1s - loss: 0.0956 - accuracy: 0.8465 - val_loss: 0.1918 - val_accuracy: 0.5099
Epoch 6/10
3035/3035 - 1s - loss: 0.0809 - accuracy: 0.8738 - val_loss: 0.2102 - val_accuracy: 0.5125
Epoch 7/10
3035/3035 - 1s - loss: 0.0620 - accuracy: 0.9074 - val_loss: 0.1586 - val_accuracy: 0.5942
Epoch 8/10
3035/3035 - 1s - loss: 0.0553 - accuracy: 0.9203 - val_loss: 0.1357 - val_accuracy: 0.7444
Epoch 9/10
3035/3035 - 1s - loss: 0.0415 - accuracy: 0.9404 - val_loss: 0.1361 - val_accuracy: 0.7457
Epoch 10/10
3035/3035 - 1s - loss: 

In [69]:
np.mean(kim_scores)

0.6962500126109302

##GRU_CNN

In [70]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
grucnn_scores = []
grucnn_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_gru_cnn()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  #grucnn_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  grucnn_scores.append(f1)
  tf.keras.backend.clear_session()

Train on 3035 samples, validate on 759 samples
Epoch 1/10
3035/3035 - 5s - loss: 0.1720 - accuracy: 0.5522 - val_loss: 0.1669 - val_accuracy: 0.5982
Epoch 2/10
3035/3035 - 1s - loss: 0.1599 - accuracy: 0.6511 - val_loss: 0.1492 - val_accuracy: 0.6825
Epoch 3/10
3035/3035 - 1s - loss: 0.1504 - accuracy: 0.6791 - val_loss: 0.1350 - val_accuracy: 0.7325
Epoch 4/10
3035/3035 - 1s - loss: 0.1371 - accuracy: 0.7265 - val_loss: 0.1362 - val_accuracy: 0.6785
Epoch 5/10
3035/3035 - 1s - loss: 0.1298 - accuracy: 0.7367 - val_loss: 0.1203 - val_accuracy: 0.7681
Epoch 6/10
3035/3035 - 1s - loss: 0.1253 - accuracy: 0.7526 - val_loss: 0.1195 - val_accuracy: 0.7866
Epoch 7/10
3035/3035 - 1s - loss: 0.1179 - accuracy: 0.7763 - val_loss: 0.1168 - val_accuracy: 0.7866
Epoch 8/10
3035/3035 - 1s - loss: 0.1124 - accuracy: 0.7954 - val_loss: 0.1145 - val_accuracy: 0.7813
Epoch 9/10
3035/3035 - 1s - loss: 0.1051 - accuracy: 0.8138 - val_loss: 0.1087 - val_accuracy: 0.8155
Epoch 10/10
3035/3035 - 1s - loss: 

In [71]:
np.mean(grucnn_scores)

0.821623196296424

#**Result**

# Result

In [72]:
gru_scores=np.average(np.array(gru_scores))
atten_scores=np.average(np.array(atten_scores))
tcn_scores=np.average(np.array(tcn_scores))
kim_scores=np.average(np.array(kim_scores))
gru_scores1=np.average
(np.array(gru_scores1))
grucnn_scores=np.mean(np.array(grucnn_scores))


In [73]:
print('f1 score of count vec' ,cv_score)
print('f1 score of word tfidf' ,tfw_score)
print('f1 score of char tfidf' ,tfc_score)

print('f1 score of RNN' ,gru_scores)
print('f1 score of gated attention',atten_scores )
print('f1 score of tcn',tcn_scores)
print('f1 score of kim',kim_scores)
print('f1 score of gru_1',gru_scores1)
print('f1 score of gru_cnn',grucnn_scores)


f1 score of count vec 0.8540543614027971
f1 score of word tfidf 0.845548565812291
f1 score of char tfidf 0.8540530522045847
f1 score of RNN 0.7934497667600902
f1 score of gated attention 0.794886579582516
f1 score of tcn 0.8145217588362555
f1 score of kim 0.6962500126109302
f1 score of gru_1 <function average at 0x7fb66455eb70>
f1 score of gru_cnn 0.821623196296424


In [ ]:
kim_test_avg=np.mean(np.array(kim_test),axis=0)
gru_test_avg=np.mean(np.array(gru_test),axis=0)
atten_test_avg=np.mean(np.array(atten_test),axis=0)
tcn_test_avg=np.mean(np.array(tcn_test),axis=0)
grucnn_test_avg=np.mean(np.array(grucnn_test),axis=0)
gru_test1_avg=np.mean(np.array(gru_test1),axis=0)


In [ ]:
kim_test[0].shape,cv_test.shape,kim_test_avg.shape

((3887,), (3887,), (3887,))

In [ ]:
print(cv_test[5:15].round())
print(tfw_test[5:15].round())
print(tfc_test[5:15].round())
print(gru_test_avg[5:15].round())
print(atten_test_avg[5:15].round())
print(kim_test_avg[5:15].round())
print(tcn_test_avg[5:15].round())
print(grucnn_test_avg[5:15].round())
print(gru_test1_avg[5:15].round())


[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]


# Ensemble

we will choose only those, having f1 greater than 0.7. Once they are selected, we will decode labels, and keep label according to mod

In [ ]:
wt=gru_scores+atten_scores+cv_score+tfw_score+tfc_score+tcn_scores+kim_scores+gru_scores1+grucnn_scores
wt

6.563347387547225

In [ ]:
#i am bit changing the distribution in order to give more weightage to ml than dl
y_test=gru_scores/wt * gru_test_avg + atten_scores/wt * atten_test_avg + cv_score/wt * cv_test + tfw_score/wt *tfw_test + tfc_score/wt *tfc_test +kim_scores/wt * kim_test_avg + tcn_scores/wt *tcn_test_avg + gru_scores1/wt *gru_test1_avg +grucnn_scores/wt*grucnn_test_avg

In [ ]:
y_test[5:15].round()

array([0., 1., 0., 0., 0., 0., 0., 0., 1., 0.])

In [ ]:
np.save('/content/drive/My Drive/dataset/OffenseEval2020/data/English/olid_ensemble.npy',y_test)

In [ ]:
def decode(ids,y_test):
  y_test[y_test>0.5]=1
  y_test[y_test<0.5]=0
  y_test=y_test.astype('int16').ravel()

  y_test=le.inverse_transform(y_test)
  y_test=pd.DataFrame(y_test,columns=['label'])
  y_test=pd.concat([ids, y_test['label']], axis=1)
  return y_test

In [ ]:
y_test=decode(ids,y_test)

In [ ]:
y_test.head()

,id,label
0,A0,OFF
1,A1,OFF
2,A2,NOT
3,A3,NOT
4,A4,NOT


# Submit file

In [ ]:
y_test.to_csv('/content/disaster.csv',index=False,header=None)


# Reset

In [ ]:
# import keras
# keras.backend.clear_session()